<a href="https://colab.research.google.com/github/HyoJuJang/05_NLP_pm/blob/2022/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_test_hanspell_clean_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-bzuia8d9
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-bzuia8d9
     |████████████████████████████████| 131 kB 9.6 MB/s 
     |████████████████████████████████| 344 kB 41.4 MB/s 
     |████████████████████████████████| 47.3 MB 1.2 MB/s 
     |████████████████████████████████| 3.6 MB 48.8 MB/s 
     |████████████████████████████████| 1.2 MB 47.2 MB/s 
     |████████████████████████████████| 3.4 MB 39.0 MB/s 
     |████████████████████████████████| 895 kB 49.2 MB/s 
     |████████████████████████████████| 3.3 MB 43.4 MB/s 
     |████████████████████████████████| 596 kB 60.7 MB/s 
     |████████████████████████████████| 61 kB 524 kB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 8.5 MB 37.5 MB/s 
     |████████████████████████████████| 127 kB 45.1 MB/s 
  Created wheel for kobert: filename=kob

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # 한국어 단발성 대화 데이터셋 불러오기 (학습용 데이터)
# # url : https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-009

# import pandas as pd
# chatbot_data = pd.read_excel('/content/drive/MyDrive/text_data/korean_short_talk.xlsx')

In [ ]:
# 2. 한국어 특정 문자 + 번역기 처리한 데이터 불러와서 학습
import pickle 

# 열기
path = '/content/drive/MyDrive/text_data/'

with open(path+'train_hanspell_new.pickle', 'rb') as f:
    chatbot_data = pickle.load(f)

In [ ]:
len(chatbot_data)

38594

In [ ]:
print(chatbot_data['Emotion'].value_counts())

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64


텍스트 교정 및 전처리한 데이터로 학습시키기

In [ ]:
# 모델 설정을 위한 변수 전처리 
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [ ]:
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
# ['sentence', 'class'] 형태로 잘 넣어졌나 확인
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는 게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는 게 어려운데 해체를 한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그 거들은 뒤부터 미치겠어요...', '3']
['최악의 상황 중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는 것이 숙제하는 것처럼 힘든 저에게 용기나 게 해주시고 할 수 있을 것 같은 희망을 주셔서 감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼 텐데 이휘재 좀 하차하라고 전해주세요', '6']


In [ ]:
# train & test set 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=1)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

28945
9649


In [ ]:
# KoBERT 입력 데이터로 만들기
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
# 이건 나중에 최적화 값 찾아봐야할 듯
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# 토큰화 실행
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
len(dataset_train[0][0])

32

In [ ]:
dataset_train[0]

['폭스콘과 함께 폭만하려는 샤프 심히 우려가 되네~~~~~!', '0']

In [ ]:
# 토큰화 패딩 처리 후 결과값 
data_train[0]

(array([   2, 4871, 6664, 7539, 5468, 4983, 4871, 6150, 7808,  517, 6546,
        7753, 3060, 7996, 3500, 1763, 5702,  517,  463,  517,  463,  517,
         463,  517,  463,  517,  463,  517,    5,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(30, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
# torch 형식의 dataset 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정해줘야함##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_dataloader

In [ ]:
# 최종 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)

    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
# import pickle

# path = '/content/drive/MyDrive/text_data/'

# with open(path+'model_hanspell_new.pickle', 'wb') as f:
#     pickle.dump(model, f)

In [ ]:
# 열기
path = '/content/drive/MyDrive/text_data/'

with open(path+'model_hanspell_new.pickle', 'rb') as f:
    model = pickle.load(f)

In [ ]:
# 새로운 문장 테스트
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# 예측 함수 생성
# torch 데이터 타입으로 좀 더 쉽게 할 수 있을 거 같은데, 일단은 모르겠어서 하나씩 예측하는 함수로 실행
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        output = out.detach().cpu().numpy()

    return np.argmax(output)

In [ ]:
e_dict = {0 : '공포', 1 : '놀람', 2 : '분노', 3 : '슬픔',
         4 : '중립', 5 : '행복', 6 : '혐오'}

In [ ]:
test[test['label']=='6']['predict'].value_counts()

In [ ]:
test[test['label']=='6'].head(30)

In [ ]:
import pandas as pd

test = pd.DataFrame(dataset_test)
test.columns = ['sentence', 'label']
test

,sentence,label
0,너무 감사해요.,5
1,언제나 응원하고 있어요!,5
2,난 이제 그 사람 번호가 없거든요,3
3,자본주의에 재벌을 죽여야 한다고? ㅋㅋㅋㅋ,2
4,독도를 다케시마로 표기한 애플은 안 쓴다.,6
...,...,...
9644,이걸 기점으로 닭 그네 탄핵 후 구속 가자!!!,2
9645,알 바 끊길 가봐 여기들 또 몰려왔네 모두 수사해서 아르바이트 비 토해내게 해야 한다,6
9646,이상하다 못해 괴상한 대통령_ 그걸 싸고도는 새누리당도 큰 문제다.... 슬프다 슬퍼..,6
9647,조선시대 묵힌 정책을 하며 세금은 시대를 반영하며 천정부지로 올리며 서민은 구닥다리...,6


In [ ]:
test['predict'] =  test_sentence['predict']

In [ ]:
import pickle
path = '/content/drive/MyDrive/text_data/'

with open(path+'test_label_done.pickle', 'wb') as f:
    pickle.dump(test, f)

In [ ]:
path = '/content/drive/MyDrive/text_data/'

with open(path+'test_label_done.pickle', 'rb') as f:
    test = pickle.load(f)

In [ ]:
# 예측 함수 생성
# torch 데이터 타입으로 시도
def predict_set(dataset_test):

    test_acc = 0.0

    tokenizer = get_tokenizer()
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

    data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

    test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_acc += calc_accuracy(out, label)
        acc = test_acc / (batch_id+1)

    return out, label, acc

In [ ]:
9649 - 64 *150

49

In [ ]:
ee = predict_set(dataset_test)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/151 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
ee

(tensor([[ 5.1618, -1.5873, -1.3380,  3.7907, -2.9372, -1.7901, -1.7577],
         [ 0.3989,  5.2512,  0.5334, -0.9857, -2.5736, -1.2474, -1.5655],
         [-1.6568, -2.0279,  5.4993, -1.1990, -1.5165, -2.5817,  2.0006],
         [-1.5129, -1.6133, -1.1639, -1.3075,  5.9532,  0.7842, -0.1805],
         [ 3.3206,  2.5758, -2.5044, -1.5925,  2.1867, -1.5769, -2.3583],
         [ 0.1399, -1.7202, -1.6967,  5.9498, -1.6610,  0.0751,  0.4290],
         [ 2.0939, -1.0125,  3.6440, -0.0811, -3.0088, -2.1403, -0.8889],
         [-1.1995, -1.0252, -0.7745, -1.8467,  5.9215,  1.1634, -1.2449],
         [-1.5747, -0.9371, -0.7257, -0.3550, -0.7855,  7.1010, -1.1420],
         [ 5.2547, -0.6369, -1.2263, -1.4151,  0.2828, -2.0527, -1.0845],
         [ 1.3698, -3.3634, -1.4198,  4.2392,  0.1053, -1.7215,  1.3699],
         [ 3.8579,  4.0101, -0.9597, -1.9069, -1.7761, -2.4471, -1.7449],
         [ 5.9816, -1.3631, -1.1121,  0.7607, -2.4795, -1.2407, -1.4666],
         [ 0.0375,  6.2724, -0.8858, -

In [ ]:
test[test['label'] == '0'].head(50)

,sentence,label,predict
8,2교대 많이 힘든가요?,0,1
38,1명이 19명 목숨을 단시간에 빼앗아갈 수 있다니 무섭네요 ...,0,0
43,사고 나면 초음속으로 천국 가겠군요ᅲ,0,1
46,내년에 군대를 간다고 하는데 기다려도 되는 건지 ...,0,0
53,이런 것도 사랑한다고 할 수 있나요ㅜㅜ?,0,0
78,그래도 보일러 안 깔아준 것만도 다행이네,0,5
80,처음엔 그냥 생리인가 했는데...,0,0
107,어떻게 해야 할지 고민입니다.,0,0
113,시간이 지날수록 더 심해질 거 같은데..,0,0
122,"근데 휴대폰 소액결제 16,500이나 나왔어요",0,1


In [ ]:
!pip install sklearn

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import recall_score
# from pycaret.classification import *

In [ ]:
pred_label=test['predict']
label_test_y=test['label'].astype('int64')

NameError: ignored

In [ ]:
pred_label.value_counts().sum()

9649

In [ ]:
label_test_y.value_counts().sum()

9649

In [ ]:
print(accuracy_score(label_test_y , pred_label))

0.5412996165405741


In [ ]:
print(recall_score(label_test_y , pred_label, average=None))

[0.59956236 0.53221477 0.4460682  0.62686567 0.41399663 0.77861035
 0.3633014 ]


In [ ]:
f1_score(label_test_y, pred_label, labels=None, average='macro', sample_weight=None, zero_division='warn')

0.5365895666672609

In [ ]:
e_dict = {0 : '공포', 1 : '놀람', 2 : '분노', 3 : '슬픔',
         4 : '중립', 5 : '행복', 6 : '혐오'}

In [ ]:
# def predict_list(input):
#   olist = []
#   for i in range(len(input)):
#     olist.append(predict(input[i]))
#   return olist

In [ ]:
# # 댓글 데이터 적용해보기
# # 열기
# path = '/content/drive/MyDrive/text_data/'

# with open(path+'naver_cm_clean.pickle', 'rb') as f:
#     naver_comment = pickle.load(f)
    
# with open(path+'daum_cm_clean.pickle', 'rb') as f:
#     daum_comment = pickle.load(f)

In [ ]:
path = '/content/drive/MyDrive/text_data/'

with open(path+'daum_comment_new.pickle', 'rb') as f:
    daum_c = pickle.load(f)
with open(path+'naver_comment_new.pickle', 'rb') as f:
    naver_c = pickle.load(f)

In [ ]:
import pandas as pd

daum = pd.DataFrame({'sentence' : sum(daum_c,[])})
daum

,sentence
0,가해자가 촉법소년이라고 피해자가 되냐어째서 그런 해석의 나오냐죄를 지어놓고 촉법소년...
1,"촉법이라 가해자라고???? 뭔 댕댕이 소리래?청소년이 칼 들고 지르면, 칼 맞은 사..."
2,전동 킥보드 같은데.. 미성년자.. 촉법소년이면.. 당연히 무면허겠지.. 그럼 아예...
3,"12, 10세인 아이들이 전동 킥보드를 탄 게 문제의 원인 아닌가? 너무 어려서..."
4,촉법소년법부모가 돈 벌고 싶으면 어린 자식 시켜서 차에 들이 받으라고 하면 되는 법...
...,...
3817,대구도 조치 좀 하자
3818,이제 남은 무개념 조폭 같은 라이더 놈들말살 타이밍
3819,누구 머리에서 뇌에서 발생한 것인지 몰라도 사고 공화국 만들려고 환장했구나
3820,3시간 이내 공유 업체에 이동 자동차도 그렇게 해라


In [ ]:
daum['label'] = daum['sentence'].apply(predict)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
daum['label'].value_counts()

6    1112
2     856
4     750
1     444
0     403
3     177
5      80
Name: label, dtype: int64

In [ ]:
daum[daum['label']==4].head(50)

,sentence,label
3,"12, 10세인 아이들이 전동 킥보드를 탄 게 문제의 원인 아닌가? 너무 어려서...",4
17,이거는 민사 소송 가야지,4
18,전킥 못 타게 해라,4
22,경찰이 잘못 했네... 애들에게 벌을 줄 수 없다는 것이지 피해자로 둔갑하라는 것...,4
27,아이들은 죄를 못 물어도 어린애들을 위험에 처하게 한 부모는 방임으로 처벌받아야죠....,4
29,경찰 뭐 하냐전동 킥보드는 반드시 면허 필수라 무면허 운전한 거고만 13세 미만 o...,4
31,선생의 권위는 땅에 박히고 가정교육마저 안되니 이런 상황이 발생하는 듯촉법소년의 경...,4
33,저런 애들 미래가 없다떡잎부터 알아본다는 말은 진리,4
39,차량 운전자에 대해 손해배상과 교통위반한 아이들 잘못까지 부모들이 모두 책임져야 하...,4
41,학생들과 부모들 와서 차량 피해자에게 사과하고 부모가 차량 피해자 정신...,4


In [ ]:
e_dict = {0 : '공포', 1 : '놀람', 2 : '분노', 3 : '슬픔',
         4 : '중립', 5 : '행복', 6 : '혐오'}

In [ ]:
daum['emotion'] = daum['label'].map(e_dict)

In [ ]:
import pickle

path = '/content/drive/MyDrive/text_data/'

with open(path+'daum_sen_done.pickle', 'wb') as f:
    pickle.dump(daum, f)

In [ ]:
# 현재 데이터로 할수 있는거 ????

# -> 전처리한 댓글 데이터 -> pretrain된 kobert 모델에 넣어서 예측값 생성 -> 긍정 / 부정 여부 확인을 통해 부정의견의 비율이 얼마나 되는지 확인해보자